In [1]:
import pandas as pd

def read_all_sheets_from_excel(path: str) -> dict:
    xls = pd.ExcelFile(path)
    df_dict = {sheet_name: xls.parse(sheet_name) for sheet_name in xls.sheet_names}
    return df_dict

# Load the dataset
file_path = 'online_retail_II.xlsx'
dataframes = read_all_sheets_from_excel(file_path)

# Print each dataframe name
print("Dataframes in this file:", ", ".join(dataframes.keys()))

Dataframes in this file: Year 2009-2010, Year 2010-2011


In [2]:
# Load the first sheet to examine its structure
sheet_2009_2010 = dataframes['Year 2009-2010']

# Display the first few rows and summary statistics
display(sheet_2009_2010.head())
print("Summary Statistics:")
sheet_2009_2010.describe(include='all')

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Summary Statistics:


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
count,525461.0,525461,522533,525461.000000,525461,525461.000000,417534.000000,525461
unique,28816.0,4632,4681,NaN,NaN,NaN,NaN,40
top,537434.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,NaN,United Kingdom
freq,675.0,3516,3549,NaN,NaN,NaN,NaN,485852
mean,NaN,NaN,NaN,10.337667,2010-06-28 11:37:36.845017856,4.688834,15360.645478,NaN
min,NaN,NaN,NaN,-9600.000000,2009-12-01 07:45:00,-53594.360000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,2010-03-21 12:20:00,1.250000,13983.000000,NaN
50%,NaN,NaN,NaN,3.000000,2010-07-06 09:51:00,2.100000,15311.000000,NaN
75%,NaN,NaN,NaN,10.000000,2010-10-15 12:45:00,4.210000,16799.000000,NaN
max,NaN,NaN,NaN,19152.000000,2010-12-09 20:01:00,25111.090000,18287.000000,NaN


In [8]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Filter out negative or zero quantities and missing descriptions
cleaned_df = sheet_2009_2010[(sheet_2009_2010['Quantity'] > 0) & (sheet_2009_2010['Description'].notnull())]

# Group by Invoice and list all items
transactions = cleaned_df.groupby('Invoice')['Description'].apply(list)

# Convert all items in transactions to strings to avoid the TypeError
cleaned_transactions = transactions.apply(lambda x: [str(i) for i in x])

# Re-apply the Transaction Encoder with the cleaned transactions
ecoder = TransactionEncoder()
transactions_encoded = encoder.fit(cleaned_transactions).transform(cleaned_transactions)
transactions_df = pd.DataFrame(transactions_encoded, columns=encoder.columns_)

# Re-applying Apriori with the cleaned data
frequent_itemsets = apriori(transactions_df, min_support=0.02, use_colnames=True)

# Re-generating Association Rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)

# Displaying the top 10 association rules based on confidence
rules.sort_values('confidence', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,(SWEETHEART CERAMIC TRINKET BOX),(STRAWBERRY CERAMIC TRINKET BOX),0.047660,0.078084,0.037899,0.795205,10.183955,0.034178,4.501648
33,(RED HANGING HEART T-LIGHT HOLDER),(WHITE HANGING HEART T-LIGHT HOLDER),0.050755,0.157882,0.036709,0.723265,4.581039,0.028696,3.043043
47,(WOODEN PICTURE FRAME WHITE FINISH),(WOODEN FRAME ANTIQUE WHITE ),0.045089,0.054278,0.028805,0.638860,11.770147,0.026358,2.618710
41,(VINTAGE HEADS AND TAILS CARD GAME ),(VINTAGE SNAP CARDS),0.035519,0.047660,0.021140,0.595174,12.487957,0.019447,2.352469
0,(72 SWEETHEART FAIRY CAKE CASES),(60 TEATIME FAIRY CAKE CASES),0.041470,0.063610,0.022473,0.541906,8.519198,0.019835,2.044100
14,(LOVE BUILDING BLOCK WORD),(HOME BUILDING BLOCK WORD),0.051945,0.063372,0.027853,0.536205,8.461247,0.024561,2.019489
11,(HEART OF WICKER SMALL),(HEART OF WICKER LARGE),0.048993,0.052183,0.026234,0.535471,10.261409,0.023678,2.040384
27,(LUNCH BAG PINK RETROSPOT),(LUNCH BAG RED SPOTTY),0.038661,0.057992,0.020521,0.530788,9.152828,0.018279,2.007640
46,(WOODEN FRAME ANTIQUE WHITE ),(WOODEN PICTURE FRAME WHITE FINISH),0.054278,0.045089,0.028805,0.530702,11.770147,0.026358,2.034764
8,(CHOCOLATE HOT WATER BOTTLE),(HOT WATER BOTTLE TEA AND SYMPATHY),0.044184,0.048374,0.022806,0.516164,10.670264,0.020669,1.966835
